### Imports

In [1]:
from IPython.display import display_html, clear_output
from itertools import chain,cycle
from copy import deepcopy
import urllib.request
import transformers
import numpy as np
import json
import time
import os
import torch
import random 
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import GroupShuffleSplit
from datasets import *
from transformers import AutoTokenizer, PreTrainedTokenizerFast, EncoderDecoderModel, Seq2SeqTrainingArguments, Seq2SeqTrainer, AdamW, DataCollatorForSeq2Seq
#from allennlp_models.rc.tools import squad

import plotly.express as px

# Display dataframes
def display(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:left"><td style="vertical-align:top">'
        html_str+=f'<h4 style="text-align: left;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)
    
# Setting seeds for reproducibility
def set_reproducibility(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    transformers.set_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # use the GPU
else:
    device = torch.device("cpu")  # use the CPU

print("Using device:", device)

Using device: cuda


### Dataset Download

In [3]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)
        
def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def download_data(data_path, url_path, suffix):    
    if not os.path.exists(data_path):
        os.makedirs(data_path)
        
    data_path = os.path.join(data_path, f'{suffix}.json')

    if not os.path.exists(data_path):
        print(f"Downloading CoQA {suffix} data split... (it may take a while)")
        download_url(url=url_path, output_path=data_path)
        urllib.request.urlretrieve(url_path, filename=data_path)
        print("Download completed!")

In [4]:
# Train data
train_url = "https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json"
download_data(data_path='coqa', url_path=train_url, suffix='train')

# Test data
test_url = "https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json"
download_data(data_path='coqa', url_path=test_url, suffix='test') 

### Preprocessing

In [5]:
# Creating Dataframes and removing unanswerable questions
train_data = json.load((open('coqa/train.json')))
test_data = json.load((open('coqa/test.json')))

qas = pd.json_normalize(train_data['data'], ['questions'], ['source', 'id', 'story'])
ans = pd.json_normalize(train_data['data'], ['answers'],['id'])
train_val_df = pd.merge(qas,ans, left_on=['id','turn_id'], right_on=['id','turn_id'])
train_val_df = train_val_df.loc[train_val_df['input_text_y']!='unknown']

qas = pd.json_normalize(test_data['data'], ['questions'], ['source', 'id', 'story'])
ans = pd.json_normalize(test_data['data'], ['answers'],['id'])
test_df = pd.merge(qas,ans, left_on=['id','turn_id'], right_on=['id','turn_id'])
test_df = test_df.loc[test_df['input_text_y']!='unknown']

In [6]:
# Removing bad turns
train_val_df = train_val_df.loc[(train_val_df['bad_turn_x'] != 'True') & (train_val_df['bad_turn_y'] != 'True')]

# Removing equal text/answer entries
train_val_df = train_val_df[train_val_df.story != train_val_df.input_text_y]
test_df = test_df[test_df.story != test_df.input_text_y]

# Removing enties with empty answers
train_val_df = train_val_df[train_val_df['input_text_y'].str.len()>0]
test_df = test_df[test_df['input_text_y'].str.len()>0]

In [7]:
# Text preprocess
def preprocess(ds,columns):
    ds = ds.replace(r'\n',' ', regex=True)
#     ds = ds.replace(r'[^\w\s]+', ' ', regex=True)
#     for feature in columns:
#         ds[feature] = ds[feature].str.lower().str.strip()
        
    return ds

columns = ['story', 'input_text_x', 'span_text', 'input_text_y']

train_val_df = preprocess(train_val_df,columns)
test_df = preprocess(test_df,columns)

In [8]:
# Train/Validation Split
set_reproducibility(42)

train_inds, val_inds = next(GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 42).split(train_val_df, groups=train_val_df['id']))

train_df = train_val_df.iloc[train_inds]
val_df = train_val_df.iloc[val_inds].reset_index()

print(train_df.columns)

Index(['input_text_x', 'turn_id', 'bad_turn_x', 'source', 'id', 'story',
       'span_start', 'span_end', 'span_text', 'input_text_y', 'bad_turn_y'],
      dtype='object')


In [9]:
# Checking the Dataframes
print(f'Training set [{train_df.shape}]')
print(f'\tFeatures: {list(train_df.columns)}')
display(train_df.loc[11:15,['id', 'input_text_x', 'input_text_y', 'span_text']])

print(f'Validation set [{val_df.shape}]')
print(f'\tFeatures: {list(val_df.columns)}')
display(val_df.loc[11:15,['id', 'input_text_x', 'input_text_y', 'span_text']])

print(f'Test set [{test_df.shape}]')
print(f'\tFeatures: {list(test_df.columns)}')
display(test_df.loc[11:15,['id', 'input_text_x', 'input_text_y', 'span_text']])

Training set [(85823, 11)]
	Features: ['input_text_x', 'turn_id', 'bad_turn_x', 'source', 'id', 'story', 'span_start', 'span_end', 'span_text', 'input_text_y', 'bad_turn_y']


,id,input_text_x,input_text_y,span_text
11,3zotghdk5ibi9cex97fepx7jetpso7,how many items are in this secret collection?,"150,000","Vatican Secret Archives were separated from the library at the beginning of the 17th century; they contain another 150,000 items."
12,3zotghdk5ibi9cex97fepx7jetpso7,Can anyone use this library?,anyone who can document their qualifications and research needs.,The Vatican Library is open to anyone who can document their qualifications and research needs.
14,3zotghdk5ibi9cex97fepx7jetpso7,what must be requested in person or by mail?,Photocopies,Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail.
15,3zotghdk5ibi9cex97fepx7jetpso7,of what books?,only books published between 1801 and 1990,hotocopies for private study of pages from books published between 1801 and 1990


Validation set [(21452, 12)]
	Features: ['index', 'input_text_x', 'turn_id', 'bad_turn_x', 'source', 'id', 'story', 'span_start', 'span_end', 'span_text', 'input_text_y', 'bad_turn_y']


,id,input_text_x,input_text_y,span_text
11,3bdcf01ogxu7zdn9vlrbf2rqzwplyf,Where was Milly led to?,Cottonwoods,led Milly Erne to Cottonwoods
12,3bdcf01ogxu7zdn9vlrbf2rqzwplyf,Who took her there?,A man,the man who had led Milly Erne to Cottonwoods
13,3bdcf01ogxu7zdn9vlrbf2rqzwplyf,Whose name would Jane not speak?,this Mormon's name,this Mormon's name
14,3bdcf01ogxu7zdn9vlrbf2rqzwplyf,Did she allow herself to even think it?,No,she did not even think it.
15,3bdcf01ogxu7zdn9vlrbf2rqzwplyf,What was Jane hoping Lassiter would become to her?,"a helper, of a friend, of a champion","the need of a helper, of a friend, of a champio"


Test set [(7917, 9)]
	Features: ['input_text_x', 'turn_id', 'source', 'id', 'story', 'span_start', 'span_end', 'span_text', 'input_text_y']


,id,input_text_x,input_text_y,span_text
11,3dr23u6we5exclen4th8uq9rb42tel,Did they want Cotton to change the color of her fur?,no,We would never want you to be any other way
12,3azhrg4cu4ktme1zh7c2ro3pn2430d,what was the name of the fish,Asta.,Asta.
13,3azhrg4cu4ktme1zh7c2ro3pn2430d,What looked like a birds belly,a bottle,a bottle
14,3azhrg4cu4ktme1zh7c2ro3pn2430d,who said that,Asta.,"""It looks like a bird's belly,"" said Asta."
15,3azhrg4cu4ktme1zh7c2ro3pn2430d,Was Sharkie a friend?,Yes,Asta's friend Sharkie


In [10]:
# Overlap Check
set_train = set(train_df['id'])
set_val = set(val_df['id'])

overlap = False
for i in set_train:
    if i in set_val:
        overlap = True
        break

print('Overlap' if overlap else 'No overlap')

No overlap


In [11]:
# Dataframes to Datasets
train_df_to_ds = train_df[columns]
val_df_to_ds = val_df[columns]
test_df_to_ds = test_df[columns]

train_df_to_ds = train_df_to_ds.rename(columns={'input_text_x': 'question', 'story': 'context',\
                                               'input_text_y': 'answer', 'span_text': 'text'})
val_df_to_ds = val_df_to_ds.rename(columns={'input_text_x': 'question', 'story': 'context',\
                                               'input_text_y': 'answer', 'span_text': 'text'})
test_df_to_ds = test_df_to_ds.rename(columns={'input_text_x': 'question', 'story': 'context',\
                                               'input_text_y': 'answer', 'span_text': 'text'})

In [12]:
# Datasets Batch split
batch_size = 8
ratio = 10

train_samples = (round(train_df_to_ds.shape[0] * ratio / 100) // batch_size) * batch_size

val_samples = (round(val_df_to_ds.shape[0] * ratio / 100) // batch_size) * batch_size
test_samples = (round(test_df_to_ds.shape[0] * ratio / 100) // batch_size) * batch_size

train_dataset = Dataset.from_dict(train_df_to_ds.iloc[:train_samples])
val_dataset = Dataset.from_dict(val_df_to_ds.iloc[:val_samples])
test_dataset = Dataset.from_dict(test_df_to_ds.iloc[:test_samples])

dataset_COQA = DatasetDict({'train':train_dataset,'validation':val_dataset,'test':test_dataset})
print(dataset_COQA)

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'text', 'answer'],
        num_rows: 34328
    })
    validation: Dataset({
        features: ['context', 'question', 'text', 'answer'],
        num_rows: 8576
    })
    test: Dataset({
        features: ['context', 'question', 'text', 'answer'],
        num_rows: 3160
    })
})


In [13]:
max_length_input = 512
max_length_answer = 42

In [14]:
def prepare_features(batch, tokenizer, max_length_input, max_length_answer):
    # Tokenize the Question and Context columns
    encoded_batch_inputs = tokenizer(
        batch['question'],
        batch['context'],
        max_length=max_length_input,
        truncation='only_second',
        padding='max_length',
        return_tensors='pt'        
    )

    # Tokenize the Answer column
    encoded_batch_labels = tokenizer(
        batch['answer'],
        max_length=max_length_answer,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    encoded_batch_inputs['labels'] = encoded_batch_labels.input_ids
#   encoded_batch_inputs['decoder_input_ids'] = deepcopy(encoded_batch_inputs['labels'])
#   encoded_batch_inputs['labels'] = [[-100 if token == tokenizer.pad_token_id else token\
#                                    for token in labels]\
#                                    for labels in encoded_batch_inputs['labels']]
    
    encoded_batch_inputs['labels_mask'] = encoded_batch_labels.attention_mask


    return encoded_batch_inputs

### Tokenization

In [15]:
model_checkpoint_M1 = 'distilroberta-base'
# Tokenizer
tokenizer_M1 = AutoTokenizer.from_pretrained(model_checkpoint_M1)
assert isinstance(tokenizer_M1, PreTrainedTokenizerFast)
tokenizer_M1.bos_token = tokenizer_M1.cls_token
tokenizer_M1.eos_token = tokenizer_M1.sep_token

# Get the special tokens and their corresponding IDs
special_tokens = tokenizer_M1.special_tokens_map
special_ids = tokenizer_M1.convert_tokens_to_ids(list(special_tokens.values()))
print("Special tokens:")
for token_type, token_list in special_tokens.items():
    print(f"{token_type}: {token_list}")
# Print the special tokens and their corresponding IDs
for token, id in zip(special_tokens.keys(), special_ids):
    print(f"{token}: {id}")

Special tokens:
bos_token: <s>
eos_token: </s>
unk_token: <unk>
sep_token: </s>
pad_token: <pad>
cls_token: <s>
mask_token: <mask>
bos_token: 0
eos_token: 2
unk_token: 3
sep_token: 2
pad_token: 1
cls_token: 0
mask_token: 50264


In [16]:
# Tokenizing the Dataset
tokenized_datasets_M1 = DatasetDict()

# Use the `prepare_features` functions
tokenized_datasets_M1 = dataset_COQA.map(
    lambda batch: prepare_features(batch, tokenizer_M1, max_length_input, max_length_answer),
    batched=True,
    batch_size=batch_size,
    remove_columns=dataset_COQA['train'].column_names
)

print(tokenized_datasets_M1)

  0%|          | 0/4291 [00:00<?, ?ba/s]

  0%|          | 0/1072 [00:00<?, ?ba/s]

  0%|          | 0/395 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'labels_mask'],
        num_rows: 34328
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'labels_mask'],
        num_rows: 8576
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'labels_mask'],
        num_rows: 3160
    })
})


### Model definition

In [17]:
# Load Model
model_M1 = EncoderDecoderModel.from_encoder_decoder_pretrained(model_checkpoint_M1, model_checkpoint_M1, tie_encoder_decoder=False)

# Model special tokens
model_M1.config.decoder_start_token_id = tokenizer_M1.cls_token_id
model_M1.config_eos_token_id = tokenizer_M1.sep_token_id
model_M1.config.pad_token_id = tokenizer_M1.pad_token_id
model_M1.config.vocab_size = model_M1.config.encoder.vocab_size

# Model hyperparams
model_M1.config.max_length = max_length_answer
model_M1.config.min_length = 1
model_M1.config.no_repeat_ngram_size = 1
model_M1.config.early_stopping = True
model_M1.config.repetition_penalty= 3.
model_M1.config.num_beams = 8

print(f"Parameters #: {model_M1.num_parameters()}")

model_M1.to(device)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['roberta.encoder.layer.1.crossattention.output.dense.bias', 'roberta.encoder.layer.2.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.5.crossatt

Parameters #: 178472025


EncoderDecoderModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

### Training

In [18]:
epochs = 3

training_args_M1 = Seq2SeqTrainingArguments(
    output_dir='./M1_Checkpoints',
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    overwrite_output_dir=True,
    #save_total_limit=2,
    fp16=True, 
    num_train_epochs = epochs,
    weight_decay=0.01,
    logging_steps=10
    #resume_from_checkpoint = True
)

# Optimizer and scheduler
optimizer_M1 = AdamW(model_M1.parameters(),lr= 5e-5)
train_steps  = epochs*len(tokenized_datasets_M1['train'])/batch_size
scheduler_M1 = transformers.get_cosine_schedule_with_warmup(optimizer=optimizer_M1,num_warmup_steps=50,num_training_steps=train_steps)
optimizers_M1 = optimizer_M1, scheduler_M1

trainer_M1 = Seq2SeqTrainer(
    model=model_M1,
    tokenizer=tokenizer_M1,
    args=training_args_M1,
    #compute_metrics=compute_metrics,
    train_dataset=tokenized_datasets_M1['train'],
    eval_dataset=tokenized_datasets_M1['validation'],
    optimizers=optimizers_M1,
    data_collator=DataCollatorForSeq2Seq(tokenizer_M1,model=model_M1)
)



C:\Users\the_s\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Using cuda_amp half precision backend


In [19]:
os.environ["WANDB_DISABLED"] = "true"

trainer_M1.train()

The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: labels_mask. If labels_mask are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 34328
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12873
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


C:\Users\the_s\anaconda3\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss
1,0.493800,0.486899
2,0.465600,0.465140
3,0.281400,0.416878


Saving model checkpoint to ./M1_Checkpoints\checkpoint-500
Configuration saved in ./M1_Checkpoints\checkpoint-500\config.json
Model weights saved in ./M1_Checkpoints\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ./M1_Checkpoints\checkpoint-500\tokenizer_config.json
Special tokens file saved in ./M1_Checkpoints\checkpoint-500\special_tokens_map.json
C:\Users\the_s\anaconda3\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./M1_Checkpoints\checkpoint-1000
Configuration saved in .

Saving model checkpoint to ./M1_Checkpoints\checkpoint-5000
Configuration saved in ./M1_Checkpoints\checkpoint-5000\config.json
Model weights saved in ./M1_Checkpoints\checkpoint-5000\pytorch_model.bin
tokenizer config file saved in ./M1_Checkpoints\checkpoint-5000\tokenizer_config.json
Special tokens file saved in ./M1_Checkpoints\checkpoint-5000\special_tokens_map.json
C:\Users\the_s\anaconda3\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./M1_Checkpoints\checkpoint-5500
Configuration saved

Saving model checkpoint to ./M1_Checkpoints\checkpoint-9500
Configuration saved in ./M1_Checkpoints\checkpoint-9500\config.json
Model weights saved in ./M1_Checkpoints\checkpoint-9500\pytorch_model.bin
tokenizer config file saved in ./M1_Checkpoints\checkpoint-9500\tokenizer_config.json
Special tokens file saved in ./M1_Checkpoints\checkpoint-9500\special_tokens_map.json
C:\Users\the_s\anaconda3\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./M1_Checkpoints\checkpoint-10000
Configuration save

TrainOutput(global_step=12873, training_loss=0.5037880903596803, metrics={'train_runtime': 2696.9615, 'train_samples_per_second': 38.185, 'train_steps_per_second': 4.773, 'total_flos': 3.178662528577536e+16, 'train_loss': 0.5037880903596803, 'epoch': 3.0})

In [24]:
# Initialize the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer_M1, model=model_M1)

# Create a DataLoader for the dataset using the data collator
test_loader = torch.utils.data.DataLoader(tokenized_datasets_M1['test'], 
                                          batch_size=batch_size, 
                                          collate_fn=data_collator)
torch.cuda.empty_cache()
# Generate answersù
i=0
for batch in tqdm(test_loader):
    
    example = batch['input_ids'].to(device)
    att_mask = batch['attention_mask'].to(device)
    generated_ids = model_M1.generate(input_ids=example, 
                                      attention_mask=att_mask,
                                      max_length=max_length_answer
                                     )
    ex = tokenizer_M1.batch_decode(example, skip_special_tokens=True)
    print(ex[0])
    print(dataset_COQA['test']['question'][i:i+batch_size])
    
    generated_answers = tokenizer_M1.batch_decode(generated_ids, skip_special_tokens=True)
    print(f'Generated ans: {generated_answers}')
    true = batch["labels"]
    ground_truth = tokenizer_M1.batch_decode(true, skip_special_tokens=True)
    print(f'True ans: {ground_truth}')
    i+=batch_size
    
    



  0%|▏                                                                                 | 1/395 [00:00<02:59,  2.20it/s]

What color was Cotton?Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing.   "What are you doing, Cotton?!"   "I only wanted to be more like you".   Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pretty and special, like you. We would neve

  1%|▍                                                                                 | 2/395 [00:00<02:24,  2.72it/s]

What did Cotton's mother and siblings do when they saw her painted orange?Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing.   "What are you doing, Cotton?!"   "I only wanted to be more like you".   Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your f

  1%|▌                                                                                 | 3/395 [00:01<02:08,  3.06it/s]

...

Did he get the job?"Everything happens for the best," my mother said whenever I was disappointed. "If you go on, one day something good will happen." When I graduated from college, I decided to try for a job in a radio station and then work hard to become a sports announcer. I took a taxi to Chicago and knocked on the door of every station, but I was t _ every time because I didn't have any working experience. Then, I went back home. My father said Montgomery Ward wanted a sports-man to help them. I applied, but I didn't get the job, either. I was very disappointed."Everything happens for the best," Mom reminded me. Dad let me drive his car to look for jobs. I tried WOC Radio in Davenport, Iowa. The program director, Peter MacArthur, told me they had already had an announcer. His words made me disappointed again. After leaving his office, I was waiting for the elevator when I heard MacArthur calling after me, "What did you say about sports? Do you know anything about football?" Then he

100%|███████████████████████████████████████████████████████████████████████████████▊| 394/395 [02:12<00:00,  2.85it/s]

Was it Peter?"Everything happens for the best," my mother said whenever I was disappointed. "If you go on, one day something good will happen." When I graduated from college, I decided to try for a job in a radio station and then work hard to become a sports announcer. I took a taxi to Chicago and knocked on the door of every station, but I was t _ every time because I didn't have any working experience. Then, I went back home. My father said Montgomery Ward wanted a sports-man to help them. I applied, but I didn't get the job, either. I was very disappointed."Everything happens for the best," Mom reminded me. Dad let me drive his car to look for jobs. I tried WOC Radio in Davenport, Iowa. The program director, Peter MacArthur, told me they had already had an announcer. His words made me disappointed again. After leaving his office, I was waiting for the elevator when I heard MacArthur calling after me, "What did you say about sports? Do you know anything about football?" Then he asked

100%|████████████████████████████████████████████████████████████████████████████████| 395/395 [02:13<00:00,  2.97it/s]

what did Cy climb?CHAPTER XXX   FERN Mullins rushed into the house on a Saturday morning early in September and shrieked at Carol, "School starts next Tuesday. I've got to have one more spree before I'm arrested. Let's get up a picnic down the lake for this afternoon. Won't you come, Mrs. Kennicott, and the doctor? Cy Bogart wants to go--he's a brat but he's lively."   "I don't think the doctor can go," sedately. "He said something about having to make a country call this afternoon. But I'd love to."   "That's dandy! Who can we get?"   "Mrs. Dyer might be chaperon. She's been so nice. And maybe Dave, if he could get away from the store."   "How about Erik Valborg? I think he's got lots more style than these town boys. You like him all right, don't you?"   So the picnic of Carol, Fern, Erik, Cy Bogart, and the Dyers was not only moral but inevitable.   They drove to the birch grove on the south shore of Lake Minniemashie. Dave Dyer was his most clownish self. He yelped, jigged, wore Car